In [ ]:
# Install necessary packages
get_ipython().system(u'pip install nilearn')
get_ipython().system(u'pip install tf-explain')

In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from nilearn import datasets  # Add this line
from nilearn.image import smooth_img
from tf_explain.core import GradCAM
import matplotlib.pyplot as plt
import pickle

In [ ]:
# Fetch dataset
n_subjects = 416
oasis_dataset = datasets.fetch_oasis_vbm(n_subjects=n_subjects)
gray_matter_map_filenames = oasis_dataset.gray_matter_maps

cdr = oasis_dataset.ext_vars['cdr'].astype(float)
cdr_numpy_arr = np.array(cdr)
cdr_numpy_arr[np.isnan(cdr_numpy_arr)] = 1
cdr_numpy_arr[cdr_numpy_arr > 0.0] = 1

In [ ]:
imgArr = []
for imgUrl in gray_matter_map_filenames:
    result_img = smooth_img(imgUrl, fwhm=1)
    imgArr.append(result_img.get_fdata())

rshapedImgArr = []
for img in imgArr:
    newImg = [cv2.resize(each_slice, (50, 50)) for each_slice in img]  # Reducing slice count
    newImg = np.array(newImg)
    rshapedImgArr.append(newImg)

label = to_categorical(cdr_numpy_arr, 2)

much_data = []
for num, img in enumerate(rshapedImgArr):
    much_data.append([img, label[num]])

In [ ]:
# Define the convolutional neural network model
IMG_SIZE_PX_X = 50
IMG_SIZE_PX_Y = 50
SLICE_COUNT = 91
n_classes = 2
batch_size = 10

model = Sequential([
    Conv3D(32, (3, 3, 3), input_shape=(IMG_SIZE_PX_X, IMG_SIZE_PX_Y, SLICE_COUNT, 1), activation='relu', padding='same'),
    MaxPooling3D(pool_size=(2, 2, 2), padding='same'),
    Conv3D(64, (3, 3, 3), activation='relu', padding='same'),
    MaxPooling3D(pool_size=(2, 2, 2), padding='same'),
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.8),
    Dense(n_classes, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Split the data into training and validation sets
train_data, validation_data = train_test_split(much_data, train_size=0.8)

# Convert the data to NumPy arrays
X_train = np.array([i[0] for i in train_data])
Y_train = np.array([i[1] for i in train_data])
X_val = np.array([i[0] for i in validation_data])
Y_val = np.array([i[1] for i in validation_data])

In [ ]:
# Train the model
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=50, batch_size=batch_size)

# Save the model
model.save("model.h5")

# Save history to a pickle file
with open("history.pkl", "wb") as file:
    pickle.dump(history, file)

In [ ]:
# Try loading history from the pickle file
try:
    with open("history.pkl", "rb") as file:
        history = pickle.load(file)
except:
    pass

# Write epoch-wise history to "Output.txt" file
with open("Output.txt", "w") as file:
    file.write("Epoch\tLoss\tAccuracy\n")
    for epoch in range(1, 51):  # Assuming 50 epochs
        loss = history.history['loss'][epoch-1]
        accuracy = history.history['accuracy'][epoch-1]
        val_loss = history.history['val_loss'][epoch-1]
        val_accuracy = history.history['val_accuracy'][epoch-1]

        file.write(f"{epoch}\t{loss}\t{accuracy}\t{val_loss}\t{val_accuracy}\n")

In [ ]:
# Plotting epoch-wise accuracy
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Epoch-wise Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('Accuracy.png')
plt.show()

# Plotting epoch-wise loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Epoch-wise Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('Loss.png')
plt.show()

In [ ]:
# Grad-CAM Visualization
index = 0  # Choose an index from the validation set
image = X_val[index][np.newaxis, ...]
class_index = np.argmax(Y_val[index])

In [ ]:
# Try loading the saved model
try:
    model = tf.keras.models.load_model("model.h5")
except:
    pass

# Create Grad-CAM explainer
explainer = Gradcam()

# Apply explainer to the model
grid = explainer.explain(([image], None), model, class_index=class_index)

In [ ]:
# Plot the original image and Grad-CAM heatmap
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image[0, :, :, SLICE_COUNT // 2, 0], cmap='gray')
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(grid[0], cmap='jet', alpha=0.8)
plt.imshow(image[0, :, :, SLICE_COUNT // 2, 0], cmap='gray', alpha=0.2)
plt.title('Grad-CAM Heatmap')

# Save the GradCAM image
plt.savefig('gradcam_image.png')

plt.show()